<a href="https://colab.research.google.com/github/michealman114/GAN-Experiments/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [2]:
"""
things to do for GAN
generator
discriminator

loss
wasserstein loss
loss for discrimnator: d(x) - d(g(r)) where x are real inputs and r is random noise fed into the generator

loss for generator: d(g(r)) 

discriminator tries to maximize the difference between values for generated and real inputs

generator tries to maximize the value of generated inputs from the discriminator
"""

def discrim_loss(results, labels):
  """
  parameters
  loss = average of d(g(r)) - d(x) (negative for minimization)
  results: batch_size * 1 tensor
  labels: batch_size tensor of labels where 1 is real and 0 is generated

  returns:
  1 dimensional tensor containing the loss
  """
  real_labels = labels.view(-1, 1)
  fake_labels = (1 - labels).view(-1,1)
  real = torch.sum(real_labels)
  fake = torch.sum(fake_labels)
  #print(real_labels)
  #print(fake_labels)
  return torch.sum(results * fake_labels) / fake - torch.sum(results * real_labels) / real

  pass

def gen_loss(results, labels):
  labels = 1 - labels
  labels = labels.view(-1, 1)
  gen_labels = torch.sum(labels)
  return torch.sum(results * labels) / -gen_labels

In [3]:
# testing of discrim loss
results = torch.rand(5,1)
labels = torch.tensor([1,1,0,0,0])
print(results)
print(labels)
print("======")
loss = discrim_loss(results, labels)
print(loss)
print("======")
print(gen_loss(results, labels))

tensor([[0.0610],
        [0.2717],
        [0.6023],
        [0.5881],
        [0.2361]])
tensor([1, 1, 0, 0, 0])
tensor(0.3092)
tensor(-0.4755)


In [27]:
import torch.nn as nn

class Gen(nn.Module):
  # GAN takes an input of noise of size 10 and constructs a 7 bit number in the form of a 7 element tensor
  def __init__(self):
    super(Gen, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(10,10),
        nn.ReLU(),
        nn.Linear(10,7),
        nn.Sigmoid()
    )
  def forward(self, input):
    return self.net(input)

In [5]:
class Discrim(nn.Module):
  def __init__(self):
    super(Discrim, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(7,7),
        nn.ReLU(),
        nn.Linear(7,1),
        nn.Sigmoid()
    )
  def forward(self, input):
    return self.net(input)

In [6]:
import random
def generate_real(num = 20):
  inputs = [[int(x) for x in format(2*random.randint(0,63), "07b")] for x in range(num)]
  
  return torch.tensor(inputs)

generate_real()

tensor([[1, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 1, 0],
        [1, 1, 0, 1, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0],
        [1, 0, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 0],
        [1, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0],
        [1, 1, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [1, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 0],
        [1, 1, 1, 0, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
        [1, 1, 0, 1, 0, 1, 0]])

In [62]:
from torch import optim
from sklearn.utils import shuffle
loss_fn = nn.BCELoss()

gen1 = Gen()
discrim1 = Discrim()

def train(num_epochs = 1000):
  gen_optim = optim.Adam(gen1.parameters(), lr = 0.001)
  discrim_optim = optim.Adam(discrim1.parameters(), lr = 0.001)

  for i in range(num_epochs):
    noise = torch.rand(200, 10)
    generated = gen1.forward(noise)
    real = generate_real(200).float()
    #data = torch.cat((generated, real)).float()
    #print("data",data.shape)
    gen_labels = torch.zeros(200,1)
    real_labels = torch.ones(200,1)
    #print("labels", labels.shape)

    #shuffled_data, shuffled_labels = shuffle(data,labels)

    #shuffled_labels = torch.flatten(shuffled_labels)

    gen_out = discrim1.forward(generated)

    #print(out.shape)

    gen_optim.zero_grad()
    loss1 = loss_fn(gen_out, real_labels)
    loss1.backward()
    gen_optim.step()

    discrim_gen_out = discrim1.forward(generated.detach())

    discrim_optim.zero_grad()
    loss2 = loss_fn(discrim_gen_out, gen_labels)
    #loss2.backward()

    discrim_optim.zero_grad()
    real_out = discrim1.forward(real)
    loss3 = loss_fn(real_out, real_labels)

    discrim_training_loss = (loss2 + loss3)/2
    
    discrim_training_loss.backward()
    discrim_optim.step()

    if (i%100 == 0):
      print(i, loss1.item(), discrim_training_loss.item())
    
  
train()

0 0.5644351840019226 0.6934731602668762
100 0.6748547554016113 0.6702186465263367
200 0.6960328817367554 0.6507169008255005
300 0.7235966324806213 0.6517740488052368
400 0.8654548525810242 0.5497804284095764
500 0.6368182897567749 0.7707153558731079
600 0.801801860332489 0.6415973901748657
700 0.5669605135917664 0.7688300013542175
800 0.842871904373169 0.6338574290275574
900 0.9422860741615295 0.516861081123352


In [67]:
test_input = torch.rand(1,10)
print(test_input.numpy())
gen1.forward(test_input)

[[0.23274231 0.6236932  0.9364887  0.9460721  0.26141596 0.40734786
  0.7823916  0.01752335 0.92254037 0.93674594]]


tensor([[4.2565e-02, 9.8760e-01, 9.7553e-01, 9.7356e-01, 1.0326e-02, 1.4891e-02,
         5.0155e-05]], grad_fn=<SigmoidBackward0>)